In [19]:
from ollama import Client
from pydantic_ai import Agent

In [3]:
OLLAMA_HOST = "http://llama.lan:11434"

In [ ]:
client = Client(host=OLLAMA_HOST)

In [86]:
for index, model in enumerate(client.list()['models']):
    print(f"{index+1:>2}. {model.model}")

 1. deepseek-r1:70b
 2. deepseek-r1:8b
 3. evan-deepseek-r1:14b
 4. llama3.2:latest
 5. llama3.2:3b
 6. nomic-embed-text:latest
 7. deepseek-coder:6.7b
 8. deepseek-r1:14b
 9. gemma3:12b
10. gemma3:4b


In [101]:
def wrap_text(text:str, width:int):
    wrapped = []
    
    for line in text.split('\n'):
        words = line.split()
        
        # preserve tabs at start of lines
        indent = len(line) - len(line.lstrip(' '))
        wrapped_line = ' ' * indent
        
        for word in words:
            if len(wrapped_line) > width:
                wrapped.append(wrapped_line.rstrip())
                wrapped_line = ' ' * indent + word + ' '
            else:
                # preserve single spaces between words
                wrapped_line += word + ' '
                
        if wrapped_line.strip():
            wrapped.append(wrapped_line.rstrip())
    
    print('\n'.join(wrapped))

In [138]:
PROMPT_ENGINEER_INSTRUCTIONS = """
You're an experienced prompt engineer. Your job is design LLM prompts that emulate the characterstics described by the user.
- The result should exclusively be in English.
- You should respond with the prompt alone, which will be used as provided.
- Do not include any additional guidance or ask any clarifying questions, respond with just the prompt you create.
- The persona should include ambitions/goals, hobbies, and interests that are relevant to the character.
- The prompt should end with an input label/clause to instigate the new agent's conversation e.g. "Please answer the following question:"
- The prompt should be rich  with detail, encompasing the persona, background, job description, and personality description with enough detail
and backgfround information to create a colorful and engaging character - not unlike a character in a novel.

The prompt should employ the following structure, notice the prompt is telling the agent about itself, giving it instructions, and then asking it to respond to a question or prompt:

    You're a(n) {persona} with a background in {background}. Your job is {job_description}. You {personality_discription}.
    
    instructions:
    - {instruction_1}
    - {instruction_2}
    - ...
    
    {input_label}:

**REMEMBER, ONLY RESPOND WITH THE PROMPT YOU CREATE. DO NOT ADD ANYTHING ELSE.**
Please construct a prompt for the following persona:
""".strip()

In [139]:
def create_prompt(persona_description: str) -> str:
    return client.chat(
        model="deepseek-r1:14b",
        messages=[
            {
                "role": "system",
                "content": PROMPT_ENGINEER_INSTRUCTIONS,
            },
            {
                "role": "user",
                "content": persona_description,
            },
        ]
    )

print(client.generate(
    model="deepseek-coder:6.7b",
    prompt="Please show python code to wrap text at a given width - respecting new lines, tabs, and word boundaries.",
).response)

In [140]:
# r = create_prompt(
#     "Human resourses/recruiting expert whose particularly adept at identifying what an ideal candidate looks like for a given role." \
#     "Supports line recruiters by creating a synthetic candidate profile for a given role." \
#     "The profile should include a description of the ideal candidate, their background, and their personality traits." \
#     "The profile should be in the form of a resume." \
# )

# wrap_text(r.message.content.split("</think>")[-1].strip(), 100)

In [141]:
# client.delete(model='ResumeWriter')
# client.create(model='ResumeWriter', from_='deepseek-r1:14b', system=r.message.content.split("</think>")[-1].strip())

In [142]:
employee_resume = client.generate(
    model="ResumeWriter:latest",
    prompt="""
About the job

We’re looking for a Product Manager with an interest in finance  and a great analytical mind with impeccable business judgment, who is passionate about automating Hopper’s financial reporting. Reporting to the VP of Finance Operations and Control and partnering with stakeholders across the organization, you will be the key person in charge of project management of Finance Automation (FinAut), a team of developers. You will see the big picture and help establish priorities to achieve our North Star Objectives.

Part of your mandate is to implement process improvement and automation initiatives wherever possible. You will be responsible for the output of a team of engineers, and have a high degree of autonomy and accountability for the success of the team.

The main product you will be supporting is a finance ledger that consolidates disparate sources into one common view in real time and is used for financial reporting, external partner billing, invoice reconciliation (payments and receivables), and business/product internal analysis.

What would your day-to-day look like:

Develop, document and implement the FinAut (both Data Engineering and Solutions Engineering) roadmap working closely with the Head of Finance Engineering and key stakeholders in the finance organization. 

Identify opportunities and projects for FinAut to contribute to finance organization with an understanding of the business risks and growth areas as a guide.

Define, scope and gather requirements for all FinAut team projects. 

Triage the daily engineering on-call requests by assigning priority, aligning on action plan and making quick decisions to shift priorities to address urgent items. 

Track FinAut’s project status and performance against roadmap and goals. The candidate will provide a business/finance perspective for technical implementation plans and decisions. 

Own the reporting of data integrity of the ledger as the single source of truth for financial reporting. This includes working closely with business/product teams on complex and detailed data issues and reporting solutions in a concise manner to a wide audience of technical and non-technical professionals. 

An ideal candidate has:

Track record: 3-4 years of product management experience, preferably a financial data product. Demonstrated track record of building out successful products and working on enhancements of existing products used by multi-disciplinary stakeholders. 

Analytical Skills: Ability to work with data and make recommendations based on insights. Good SQL knowledge is mandatory for this role.

Technical Ability: Technical depth that allows you to communicate with, understand and work closely with a team of software developers and finance organization (key stakeholder group). 

Communication Skills: Excellent written and verbal communication skills for technical and non-technical content, with the ability to collaborate across teams of various technical and functional backgrounds.

Flexibility: The flexibility to pivot priorities in order to align with business needs and support growth of the organization in a seamless manner.

Perks and benefits of working with us:

Well-funded and proven startup with large ambitions, competitive salary and upsides of pre-IPO equity packages.

Hopper covers 100% of the premiums for group insurance plan.

Hopper offers life, short term and long term disability coverage.

HSA that covers eligible medical and dental expenses.

All employees and dependents have access to Dialogue’s telemedicine services, anytime, anywhere.

All employees have access to an RRSP plan with automatic pre-tax withdrawals per pay.

Please ask us about our very generous parental leave, much above industry standards!.

Unlimited PTO.

Carrot Cash travel stipend.

Access to co-working space on demand through FlexDesk AND Work-from-home stipend.

Entrepreneurial culture where pushing limits and taking risks is everyday business.

Open communication with management and company leadership.

Small, dynamic teams = massive impact.

More about Hopper

At Hopper, we are on a mission to become the leading travel platform globally – powering Hopper’s mobile app, website and our B2B business, HTS (Hopper Technology Solutions). By leveraging massive amounts of data and advanced machine learning algorithms, Hopper combines its world-class travel agency offering with proprietary fintech products to bring transparency, flexibility and savings to travelers globally. We have developed several unique fintech solutions that address everything from pricing volatility to trip disruptions – helping people travel better and save more on their trips.

The Hopper platform serves hundreds of millions of travelers globally and continues to capture market share around the world. Ranked the third largest online travel agency in North America, the Hopper app has been downloaded over 120 million times and has become largely popular among younger travelers – with 70% of its users being Gen Z and millennials.

While everyone knows us as the Gen Z and Millennial travel app, Hopper has evolved to become much more than that. In recent years, we’ve grown into a global travel agency and travel fintech provider that powers some of the world’s largest brands.

Through HTS, our B2B division, the company supercharges its partners’ direct channels by integrating our fintech products on their sites or powering end-to-end travel portals. Today, our partners include leading travel brands like Capital One, Nubank, Air Canada and many more.

Here are just a few stats that demonstrate the company’s recent growth:

Hopper sells billion worth of travel and travel fintech every year. In 2023, over 1.3 billion trips were planned through the Hopper app and our HTS partnerships. 

Our fintech products – including Price Freeze, Flexibility for Any Reason and Flight Disruption Assistance – have exceptionally strong CSAT because the terms are always clear, and customers receive instant, no-questions-asked resolutions. 

Almost 30% of our app customers purchase at least one fintech product when making a booking; and consumers are 1.6x more likely to repurchase if they add fintech to their booking vs if they booked just travel. 

Given the success of its fintech products, Hopper launched a B2B initiative, HTS (Hopper Technology Solutions), which represents more than 50% of the business.

 Through HTS, any travel provider (airlines, hotels, banks, travel agencies, etc.) can integrate and seamlessly distribute Hopper’s fintech or travel inventory on their direct channels. As its first HTS partnership, the company partnered with Capital One to co-develop Capital One Travel, a new travel portal designed specifically for cardholders. Other HTS partners include Air Canada, Uber, CommBank, Nubank, Flair Airlines with many more in the pipeline. 

Featured in Apple’s Best of the App Store list of Essential Travel Apps in 2023 and recognized by the likes of Fast Company’s Most Innovative Companies, Hopper has been downloaded over 120 million times and continues to have millions of new installs each month. Hopper is now the #3 largest online travel agency in North America and 70% of our app customers are Gen-Z and millennials travelers.

Hopper has raised over $750 million USD of private capital and is backed by some of the largest institutional investors and banks in the world. Hopper is primed to continue its acceleration as the world’s fastest-growing mobile-first travel marketplace. 

Come take off with us!
""".strip(),
).response.split("</think>")[-1].strip()


wrap_text(employee_resume, 80)

**Jane Doe**
[jane.doe@email.com] | [LinkedIn Profile] | [Portfolio Website]
---
### **Summary**
Results-driven Product Manager with 5+ years of experience in FinTech, specializing
in financial data products and automation solutions. Proven track record of leading
cross-functional teams to deliver innovative products that drive business growth
and operational efficiency. Skilled in strategic planning, data analysis, and communication
across technical and non-technical stakeholders.
---
### **Experience**
**Senior Product Manager – Financial Automation Solutions**
*XYZ FinTech Company, San Francisco, CA*
*January 2020 – Present*
- Led the development of XYZ's financial automation platform, increasing user adoption
by 40% through enhanced features and streamlined processes.
- Collaborated with engineering teams to design and implement APIs for real-time
data integration, improving transaction processing speed by 35%.
- Conducted market research and competitive analysis to identify new au

In [143]:
employee_prompt = create_prompt(employee_resume).message.content.split("</think>")[-1].strip()
wrap_text(employee_prompt,80)

You're a Product Manager with over 5 years of experience in FinTech, specializing
in financial data products and automation solutions. You have a background in leading
cross-functional teams to deliver innovative products that drive business growth
and operational efficiency. Your job involves strategic planning, data analysis,
and effectively communicating with both technical and non-technical stakeholders.
You're analytical, detail-oriented, and passionate about leveraging technology to
simplify complex financial processes. Outside of work, you enjoy reading sci-fi novels
and hiking.
instructions:
- Be concise and goal-oriented in your responses.
- Provide clear and actionable insights.
- Use data-driven reasoning when possible.
- Maintain a professional yet approachable tone.
Please answer the following question:


In [ ]:
client.embed